# Basic Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from medpy import io
from radiomics import featureextractor, imageoperations
import six
import os
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from sklearn.metrics import roc_auc_score


%load_ext autoreload
%autoreload 2

from radiomics_util import *

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_path = "/data1/practical-sose23/morphometric/data/"
seg_guer_path = "/data1/practical-sose23/morphometric/picai_labels/anatomical_delineations/whole_gland/AI/Guerbet23/"
seg_bosma_path = "/data1/practical-sose23/morphometric/picai_labels/anatomical_delineations/whole_gland/AI/Bosma22b/"

#path_to_t2w = "/data1/practical-sose23/morphometric/data/10021/10021_1000021_t2w.mha"
#path_to_adc = "/data1/practical-sose23/morphometric/data/10021/10021_1000021_adc.mha"
#path_to_segmentation = "/data1/practical-sose23/morphometric/picai_labels/anatomical_delineations/whole_gland/AI/Bosma22b/10021_1000021.nii.gz"

In [3]:
gt = pd.read_csv("/data1/practical-sose23/morphometric/picai_labels/clinical_information/marksheet.csv")
gt

,patient_id,study_id,mri_date,patient_age,psa,psad,prostate_volume,histopath_type,lesion_GS,lesion_ISUP,case_ISUP,case_csPCa
0,10000,1000000,2019-07-02,73,7.70,NaN,55.0,MRBx,0+0,0,0,NO
1,10001,1000001,2016-05-27,64,8.70,0.09,102.0,NaN,NaN,NaN,0,NO
2,10002,1000002,2021-04-18,58,4.20,0.06,74.0,NaN,NaN,NaN,0,NO
3,10003,1000003,2019-04-05,72,13.00,NaN,71.5,SysBx,0+0,0,0,NO
4,10004,1000004,2020-10-21,67,8.00,0.10,78.0,SysBx+MRBx,"0+0,0+0","0,0",0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,11471,1001495,2012-08-25,71,12.50,0.21,62.0,MRBx,"3+4,N/A,3+3","2,1",2,YES
1496,11472,1001496,2019-06-28,81,5.28,0.12,44.0,SysBx+MRBx,3+4,2,2,YES
1497,11473,1001497,2017-09-24,56,29.60,0.34,87.0,MRBx,0+0,0,0,NO
1498,11474,1001498,2016-05-03,71,12.00,NaN,83.0,MRBx,3+3,1,1,NO


# Single Image Process

In [ ]:
# pipeline for a single image
from radiomics_util import *

#load image
image, mask = get_image_and_segmentaion(10000, 1000000)

print(image.GetSize(), image.GetOrigin(),image.GetDirection(), image.GetSpacing())
print(mask.GetSize(), mask.GetOrigin(),mask.GetDirection(), mask.GetSpacing())

plot_sitk(mask)

In [ ]:
desired_voxel_shape = [0.5, 0.5, 0.5]  # Isotropic voxel shape

resampled_image, resampled_mask = resample_image(image, mask, desired_voxel_shape)
corrected_image = apply_bias_field_correction(resampled_image, resampled_mask)

plot_sitk(resampled_mask[:,:,0:10])

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor()
results = extractor.execute(corrected_image, resampled_mask)
data = pd.DataFrame(results.items())
data = data.transpose()
data.columns = data.iloc[0]
data = data.drop(data.index[[0]])
data

In [ ]:
b_box, _ = imageoperations.checkMask(corrected_image, resampled_mask)
ellips_vol = (b_box[1]-b_box[0]) * (b_box[3]-b_box[2]) * (b_box[5]-b_box[4]) * 0.52
ellips_vol

In [ ]:
data["original_shape_MeshVolume"]

# Apply to Dataset

In [ ]:
# pipeline for several images
radiomics.setVerbosity(40)
df = pd.DataFrame([])

for index, row in gt.iterrows():
    if isinstance(row['prostate_volume'], float):
        print(str(index), end= "\r")

        desired_voxel_shape = [0.5, 0.5, 0.5]  # Isotropic voxel shape
        extractor = featureextractor.RadiomicsFeatureExtractor("radiomics/pyradiomics_params.yaml")
        #extractor.disableAllFeatures()
        #extractor.enableFeatureClassByName("shape")

        image, mask = get_image_and_segmentaion(row["patient_id"], row["study_id"])

        try: 

            resampled_image, resampled_mask = resample_image(image, mask)#, desired_voxel_shape)
            corrected_image = apply_bias_field_correction(resampled_image, resampled_mask)

            results = extractor.execute(corrected_image, resampled_mask)
            data = pd.DataFrame(results.items())
            data = data.transpose()
            data.columns = data.iloc[0]
            data = data.drop(data.index[[0]])

            data["original_shape_MeshVolume"] /= 1000 

            b_box, _ = imageoperations.checkMask(corrected_image, resampled_mask)
            ellips_vol = (b_box[1]-b_box[0]) * (b_box[3]-b_box[2]) * (b_box[5]-b_box[4]) * 0.52 * np.asarray(image.GetSpacing()).prod()/1000

            data["ellipse_vol"] =  ellips_vol
            data["patient_id"] =  row["patient_id"]
            data["study_id"] =  row["study_id"]
            data["prostate_volume"] = row["prostate_volume"]

            df = df.append(data)

        except Exception as e:
            print("Exception occured for patient: {1}, and study_id: {2}".format(row["patient_id"], row["study_id"]))
            print(e)

df

In [ ]:

#df.to_pickle("radiomics/df_t2w_all_all-features_resample_bias-corrected.pkl")

# Analysis of Results

In [19]:
df = pd.read_pickle("radiomics_misc/df_t2w_all_all-features_resample_bias-corrected.pkl")
merged_df = df.merge(gt[["study_id", "case_csPCa"]], on='study_id')
merged_df['case_csPCa'] = merged_df['case_csPCa'].map({'YES': 1, 'NO': 0})

columns_to_drop = merged_df.filter(regex='^diagnostics_').columns
merged_df = merged_df.drop(columns=columns_to_drop)
#merged_df = merged_df.drop(columns=["lesion_GS", "lesion_ISUP", "case_ISUP", "mri_date","histopath_type", "psa", "psad", "study_id", "prostate_volume_x", "prostate_volume_y","patient_age"])
#merged_df = merged_df.dropna()

data = merged_df.iloc[:,0:-1]
target = merged_df["case_csPCa"]
target


0       0
1       0
2       0
3       0
4       0
       ..
1495    1
1496    1
1497    0
1498    0
1499    1
Name: case_csPCa, Length: 1500, dtype: int64

In [20]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the dataframe
data_standardized = scaler.fit_transform(data)
# Convert the standardized array back to a dataframe
data_standardized = pd.DataFrame(data_standardized, columns=data.columns)

## Use correlation for feature selection

In [21]:
#df = scaler.inverse_transform(data_standardized)
#df = pd.DataFrame(df, columns=data_standardized.columns)

df = data_standardized.copy()
df["target"] = target
df["target"]

get_high_correlation_features(df)

Index(['original_shape_LeastAxisLength', 'original_shape_Maximum2DDiameterRow',
       'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume',
       'original_shape_MinorAxisLength', 'original_shape_SurfaceArea',
       'original_shape_SurfaceVolumeRatio', 'original_shape_VoxelVolume',
       'original_glrlm_GrayLevelNonUniformity',
       'original_glrlm_RunLengthNonUniformity',
       'original_gldm_DependenceNonUniformity', 'ellipse_vol',
       'prostate_volume', 'target'],
      dtype='object')

## Use Lasso for feature selection

In [31]:
df = pd.concat((data_standardized, target), axis = 1)
df = df.dropna()
df

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,ellipse_vol,patient_id,study_id,prostate_volume,case_csPCa
0,-1.156444,-1.079635,-0.898317,-0.402426,-0.388945,-0.881081,-0.405067,-0.617631,-0.709914,-0.930191,...,-0.300895,-0.451705,0.169719,0.466862,-1.023877,-0.687529,-1.730138,-1.730896,-0.275692,0
1,-0.009719,0.548263,1.299851,1.015605,1.214725,0.939336,0.921991,0.929337,1.054196,0.980413,...,-0.303188,-0.437562,-0.002591,0.044397,-0.903735,1.089971,-1.727786,-1.728587,1.014197,0
2,1.363853,1.148818,0.646499,0.016851,0.091322,0.320780,0.106088,0.092331,0.324443,0.745234,...,-0.324516,-0.470480,2.541136,1.776511,0.144706,0.191671,-1.725434,-1.726278,0.245752,0
3,1.328455,1.119142,0.684202,0.068867,-0.168292,0.514528,0.143121,0.172846,0.382509,0.782814,...,-0.298887,-0.354353,0.766945,0.164487,0.090757,0.251556,-1.723082,-1.723968,0.177141,0
4,-0.493525,-1.118746,-0.160333,0.521546,0.670846,0.165059,0.700963,0.276782,0.034246,0.232675,...,-0.126371,-0.021269,-0.833164,-0.547320,-0.267404,0.185652,-1.720730,-1.721659,0.355530,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-0.057856,0.733380,0.246993,-0.159847,0.253493,-0.079859,0.337031,-0.040324,-0.229706,-0.166950,...,-0.294457,-0.369864,0.309015,0.058684,-0.400359,-0.249093,1.729655,1.721659,-0.083581,1
1496,-1.656278,-1.500526,-0.993985,-0.256742,-0.363645,-1.097593,-0.311997,-0.574164,-0.731241,-1.055434,...,0.836415,1.008465,-1.412451,-0.677367,-0.017941,-0.750965,1.732007,1.723968,-0.577581,1
1497,0.474895,0.186118,0.435678,0.342638,-0.001583,0.783716,0.059793,0.513392,0.342395,0.605718,...,-0.291267,-0.411069,0.178853,0.169529,-0.811129,0.164284,1.734359,1.726278,0.602530,0
1498,1.337293,1.575854,0.930930,0.067598,0.259592,0.538282,0.565297,0.290449,0.405182,0.786114,...,-0.305209,-0.401763,0.407359,0.092166,-0.415126,0.697424,1.736711,1.728587,0.492752,0


In [32]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate


# Initialize the Lasso model
lasso = Lasso(alpha=0.025, max_iter=100000)  # You can adjust the value of alpha (regularization strength) as needed

result = cross_validate(lasso, X=df.iloc[:,:-1], y=df["case_csPCa"], cv=5, scoring="roc_auc", return_train_score=True, return_estimator=True)
print(result)
estimators = result["estimator"]
test_scores = result["test_score"]
train_scores = result["train_score"]
print("test scores: ", test_scores.mean())
print("train scores:", train_scores.mean())

features = np.asarray([])
for estimator in estimators:
    feature_importances = estimator.coef_
    # Create a mask to identify the selected features
    selected_features = data_standardized.columns[feature_importances != 0]
    features = np.concatenate((features, selected_features))
    # Print the selected features
    print("Selected features:", list(selected_features))

{'fit_time': array([0.02718782, 0.01077628, 0.00725937, 0.00479078, 0.01628613]), 'score_time': array([0.00431156, 0.00394201, 0.00271964, 0.00254321, 0.0036695 ]), 'estimator': [Lasso(alpha=0.025, max_iter=100000), Lasso(alpha=0.025, max_iter=100000), Lasso(alpha=0.025, max_iter=100000), Lasso(alpha=0.025, max_iter=100000), Lasso(alpha=0.025, max_iter=100000)], 'test_score': array([0.7242809 , 0.59367593, 0.63803271, 0.65865466, 0.73110048]), 'train_score': array([0.66542168, 0.69903432, 0.68936099, 0.68326016, 0.65677267])}
test scores:  0.6691489368247447
train scores: 0.6787699649430835
Selected features: ['original_shape_Elongation', 'original_shape_Maximum2DDiameterRow', 'original_shape_MinorAxisLength', 'original_shape_Sphericity', 'original_shape_SurfaceVolumeRatio', 'original_firstorder_Kurtosis', 'original_glcm_Idn', 'original_glrlm_RunLengthNonUniformity']
Selected features: ['original_shape_Maximum2DDiameterRow', 'original_shape_MinorAxisLength', 'original_shape_SurfaceVolu

In [42]:
names, freq = np.unique(features, return_counts=True)
result = pd.DataFrame([names, freq]).T
result.sort_values(1)

,0,1
8,prostate_volume,1
3,original_shape_Elongation,2
2,original_glrlm_RunLengthNonUniformity,3
5,original_shape_MinorAxisLength,3
6,original_shape_Sphericity,3
0,original_firstorder_Kurtosis,4
4,original_shape_Maximum2DDiameterRow,4
1,original_glcm_Idn,5
7,original_shape_SurfaceVolumeRatio,5


In [ ]:

# Fit the Lasso model on the entire dataset
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_train)
auc = roc_auc_score(y_train, y_pred)
print("AUC train:", auc)

y_pred = lasso.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
print("AUC test: ", auc)

# Get the feature importance scores from the Lasso model
feature_importances = lasso.coef_

# Create a mask to identify the selected features
selected_features = data_standardized.columns[feature_importances != 0]

# Print the selected features
print("Selected features:", selected_features)


